In [1]:
### UTILS
def get_input(dayNum):
    import os
    data_file = f"./data/day-{dayNum}.input"
    if not os.path.exists(data_file):
        download_input(dayNum) 
    return [line.strip() for line in open(data_file).readlines()]

def download_input(dayNum):
    import subprocess
    SESSION_COOKIE = open('./session_cookie.secret').readlines()[0].strip()
    url = f"https://adventofcode.com/2020/day/{dayNum}/input"
    cmd = f"curl -H 'Cookie: session={SESSION_COOKIE}' {url} > ./data/day-{dayNum}.input"
    subprocess.run(cmd, capture_output=True, shell=True)

# input is e.g. [0,1,0,0]
# output is e.g 0b0100 -> 4
def bits_to_int(bits):
    return int("0b" + "".join([str(b) for b in bits]), base=2)

assert(bits_to_int([0,1,0,0]) == 4)

In [2]:
# https://adventofcode.com/2020/day/4
def Day4():
    def get_passports():
        passports = []
        lines = get_input(4)
        cur_passport = {}
        for line in lines:
            if line == "":
                if cur_passport:
                    passports.append(cur_passport)
                cur_passport = {}
            field_pairs = line.split()
            for pair in field_pairs:
                [key,val] = pair.split(':')
                cur_passport[key] = val
        passports.append(cur_passport)
        return passports
        
    passports = get_passports()
    
    def valid_1(passport):
        required_fields = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']
        if not all([key in passport for key in required_fields]):
            return False
        return True

    def valid_2(passport):
        import re
        if not valid_1(passport):
            return False

        def validate_height(x):
            match = re.match("(\d+)(cm|in)", x)
            if match is None:
                return False
            [hgt,unit] = match.groups()
            hgt = int(hgt)
            if unit == 'cm':
                return hgt >= 150 and hgt <= 193
            elif unit == 'in':
                return hgt >= 59 and hgt <= 76
            else:
                raise f"Unexpected value for hgt {hgt} unit {unit}"
        
        validations = {
            'byr': lambda x: int(x) >= 1920 and int(x) <= 2002,
            'iyr': lambda x: int(x) >= 2010 and int(x) <= 2020,
            'eyr': lambda x: int(x) >= 2020 and int(x) <= 2030,
            'hgt': validate_height,
            'hcl': lambda x: re.match("^#[0-9a-f]{6}$",x) is not None,
            'ecl': lambda x: x in 'amb blu brn gry grn hzl oth'.split(' '),
            'pid': lambda x: re.match("^[0-9]{9}$",x) is not None,
        }
        
        for field in passport:
            if field in validations:
                if not validations[field](passport[field]):
                    return False
        return True

    def day1():
        return len([p for p in passports if valid_1(p)])
    
    def day2():
        return len([p for p in passports if valid_2(p)])
    
    return day1(), day2()

print(f"Day 4: {Day4()}")

Day 4: (190, 121)


In [3]:
# https://adventofcode.com/2020/day/5
def Day5():
    def pid(d):
        row_bits = [1 if b=='B' else 0 for b in d[:7]]
        col_bits = [1 if b=='R' else 0 for b in d[7:]]
        row = bits_to_int(row_bits)
        col = bits_to_int(col_bits)
        return row*8 + col
    
    def pids():
        return [pid(d) for d in get_input(5)]
    
    def part1():
        return max(pids())
    
    def part2():
        pids_ = pids()
        for id in sorted(pids_):
            if (id+1) not in pids_:
                return id + 1
            
    return part1(), part2()
    
print(f"Day 5: {Day5()}")

Day 5: (976, 685)


In [4]:
def Day6():
    data = get_input(6)
    
    def part1():
        groups = []
        group = set()
        for line in data:
            if line == '':
                groups.append(group)
                group = set()
            else:
                for char in line:
                    group.add(char)
            
        if group is not None:
            groups.append(group)
            
        return sum([len(g) for g in groups])
            
    def part2():
        groups = []
        group = {'len': 0}
        for line in data:
            if line == '':
                groups.append(group)
                group = {'len':0}
            else:
                group['len'] += 1
                for char in line:
                    if char not in group:
                        group[char] = 1
                    else:
                        group[char] += 1
        if group is not None:
            groups.append(group)
        
        count = 0
        for group in groups:
            g_len = group['len']
            del group['len']
            count += len([k for k in group if group[k] == g_len])
        return count
            
    
    return part1(), part2()

print(f"Day 6: {Day6()}")

Day 6: (6430, 3125)


In [5]:
def Day7(data=get_input(7)):
    def parse(l):
        l = l[:-1] # drop the '.'

        import re
        container_re = r"^(.*) bags$"
        item_re = r"^(\d+) (.*) bags?$"

        container, items = l.split(' contain ')
        container = re.match(container_re, container).groups()[0]
        items_dict = {}

        if items == 'no other bags':
            pass
        else:
            items = items.split(', ')
            for i in items:
                count,name = re.match(item_re, i).groups()
                count = int(count)
                items_dict[name] = count
        return container,items_dict

    
    def part1():
        nodes = {}
        for line in data:
            container,items = parse(line)
            if container not in nodes:
                nodes[container] = set()
            for item in items:
                if not item in nodes:
                    nodes[item] = set()
                nodes[item].add(container)
        seen = set()
        check = nodes['shiny gold']
        while len(check) > 0:
            cur = check.pop()
            if cur not in seen:
                seen.add(cur)
                check |= nodes[cur]
                
        return len(seen)
            
    def part2():
        
        nodes = {}
        for line in data:
            container,items = parse(line)
            assert(container not in nodes)
            nodes[container] = items
        
        def cost(node):
            return sum([count + count*cost(child) for child,count in nodes[node].items()])
        
        return cost('shiny gold')
                    
    return part1(),part2()

ex1 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".split("\n")
ex2 = """shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.""".split("\n")

assert(Day7(ex1) == (4,32))
assert(Day7(ex2)[1] == 126)

print(f"Day 7: {Day7()}")

Day 7: (372, 8015)


In [6]:
def Day8(data = get_input(8)):
    def parse(ins):
        op,offset = ins.split(' ')
        offset = int(offset)
        return (op,offset)
        
    
    def evaluate(instructions):
        acc,pc = (0,0)
        states = []
        seen = []
        delta = 1
        
        def process(op, offset):
            nonlocal acc
            nonlocal pc

            if op == 'acc':
                acc += offset
                pc += delta
            elif op == 'nop':
                pc += delta
            elif op == 'jmp':
                pc += delta*offset
            else:
                assert(False)
        
        while True:
            if pc in seen:
                break
            states.append( (acc, pc) )
            seen.append(pc)
            if pc >= len(instructions) or pc < 0:
                break
            ins = instructions[pc]
            process(*ins)

        return states
            
    
    def part1():
        instructions = [parse(ins) for ins in data]
        states = evaluate(instructions)
        return states[-1][0]
    
    def part2():
        instructions = [parse(ins) for ins in data]
        instruction_copies = []
        for idx,(op,offset) in enumerate(instructions):
            if op in ['jmp','nop']:
                copy = instructions.copy()
                other = 'jmp' if op == 'nop' else 'nop'
                copy[idx] = (other, offset)
                instruction_copies.append(copy)
        for copy in instruction_copies:
            states = evaluate(copy)
            (acc,pc) = states[-1]
            if pc == len(copy): # completed
                return acc

        
    
    return part1(),part2()

print(f"Day 8: {Day8()}")

Day 8: (1928, 1319)


In [7]:
def Day9(size=25):
    data = [int(i) for i in get_input(9)]

    def is_error(arr, d):
        for i in range(len(arr)):
            for j in range(i+1, len(arr)):
                if arr[i] + arr[j] == d:
                    return False
        return True
    
    def part1():
        queue = []
        for d in data:
            if len(queue) < size:
                queue.append(d)
            else:
                if is_error(queue,d):
                    return d
                queue.append(d)
                queue = queue[-size:]
        assert(False)
            
    
    def sum_from(i, data, target):
        start = i
        acc = 0
        while i < len(data):
            acc += data[i]
            if acc == target:
                return data[start:i+1]
            if acc > target:
                return None
            i += 1
        return None
    
    def part2():
        target = part1()
        for i in range(len(data)):
            found = sum_from(i, data, target)
            if found is not None:
                return min(found) + max(found)

    return part1(),part2()

print(f"Day 9: {Day9()}")

Day 9: (27911108, 4023754)


In [30]:
def Day10(data=get_input(10)):
    from collections import defaultdict
    data = [int(d) for d in data]

    def part1(data):
        acc = 0
        _max = max(data)
        deltas = defaultdict(lambda: 0)
        data = sorted([0] + data + [_max + 3])
        for i in range(1, len(data)):
            delta = data[i] - data[i-1]
            deltas[delta] += 1
        return deltas[1] * deltas[3]
            
        
    return part1(data)

ex1 = [16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]
ex2 = [28,
 33,
 18,
 42,
 31,
 14,
 46,
 20,
 48,
 47,
 24,
 23,
 49,
 45,
 19,
 38,
 39,
 11,
 1,
 32,
 25,
 35,
 8,
 17,
 7,
 9,
 4,
 2,
 34,
 10,
 3]

print(f"Day 10: {Day10()}")

Day 10: 1690


In [ ]:
# POST to https://adventofcode.com/2020/day/10/answer
# Form Data level:1, answer: 1690